In [6]:
import numpy as np
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Word2Vec 

In [7]:
def word2vec(df):
    tknzr = TweetTokenizer()
    df = df['text'].apply(lambda x: tknzr.tokenize(x))
    model_train = Word2Vec(df, window=5, min_count=1, workers=4)
    model_train.train(df, total_examples= len(df), epochs=20)

    #create a vector for each tweet
    def word_vector(tokens, size):
        vector = np.zeros(size).reshape((1,size))
        count = 0
        for word in tokens:
            try:
                vector += model_train.wv[word].reshape(1,size)
                count += 1
            except KeyError:
                pass
        if count != 0:
            vector /= count
        return vector

    #apply vectorize_tweet to df
    wordvec_arrays = np.zeros((len(df), 100)) 
    for i in range(len(df)):
        wordvec_arrays[i,:] = word_vector(df[i], 100)

    wordvec_df = pd.DataFrame(wordvec_arrays)
    
    return wordvec_df


In [8]:
df_train = pd.read_csv('train_processed.csv')
df_val = pd.read_csv('val_processed.csv')

#labels for training dataset
train_labels_stances= df_train['stance'].values
train_labels_categories= df_train['category'].values

#labels for validation dataset
val_labels_stances= df_val['stance'].values
val_labels_categories= df_val['category'].values


wordvec_train = word2vec(df_train)
wordvec_val = word2vec(df_val)


# ML models for stances

In [9]:
models=[SVC(),LogisticRegression()]

for model in models:
    print(model)
    model.fit(wordvec_train, train_labels_stances)
    predicted_stance= model.predict(wordvec_val) 
    print(classification_report(val_labels_stances ,predicted_stance))
    print("---------------------------------")


SVC()


c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        70
           0       0.00      0.00      0.00       126
           1       0.80      1.00      0.89       804

    accuracy                           0.80      1000
   macro avg       0.27      0.33      0.30      1000
weighted avg       0.65      0.80      0.72      1000

---------------------------------
LogisticRegression()
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        70
           0       0.50      0.01      0.02       126
           1       0.80      1.00      0.89       804

    accuracy                           0.80      1000
   macro avg       0.43      0.34      0.30      1000
weighted avg       0.71      0.80      0.72      1000

---------------------------------


c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1344: Und

# ML models for categories

In [10]:

for model in models:
    print(model)
    model.fit(wordvec_train, train_labels_categories)
    predicted= model.predict(wordvec_val) 
    print(classification_report(val_labels_categories ,predicted))
    print('------------------------------')



SVC()


c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

              precision    recall  f1-score   support

           0       0.55      1.00      0.71       545
           1       0.00      0.00      0.00       145
           2       0.00      0.00      0.00        82
           3       0.00      0.00      0.00        20
           4       0.00      0.00      0.00        15
           5       0.00      0.00      0.00        10
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00       128
           8       0.00      0.00      0.00        36
           9       0.00      0.00      0.00        17

    accuracy                           0.55      1000
   macro avg       0.05      0.10      0.07      1000
weighted avg       0.30      0.55      0.38      1000

------------------------------
LogisticRegression()
              precision    recall  f1-score   support

           0       0.55      1.00      0.71       545
           1       0.00      0.00      0.00       145
           2       0.00   

c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1344: Und